### Build fragments

In [1]:
import pandas as pd
import json
from pathlib import Path

from tg.grammar_ru import CorpusReader
from tg.projects.book_fragments.fragments_builder import FragmentsBuilder
from tg.projects.book_fragments.ru_localizator import RussianLocalizator
from tg.projects.book_fragments.eng_localizator import EnglishLocalizator
from pathlib import Path

In [ ]:
cip_eng_corpus = Path('./files/corpuses/eng_crime_and_puhishment.base.zip')
cip_ru_corpus = Path('./files/corpuses/ru_crime_and_puhishment.base.zip')
fragments_builder = FragmentsBuilder(cip_eng_corpus, output_path="./files/fragments", file_name="eng_cip_fragments")
fragments_builder.construct_fragments_json()

In [ ]:
file = open("./files/fragments/eng_cip_fragments.json")
data = json.load(file)
file.close()

for fragment in data["fragments"]:
    input("next fragment")
    print(fragment['text'])

In [ ]:
martin_reader = CorpusReader(Path('../retell/featurized_corpuses/books/eng/Martin.featurized.books.corpus.zip'))

frame = martin_reader.get_frames().first()
# frame[frame["sentence_id"] == 9]

count_extra_sentences = 0
extra_sentences_id = []
for sentence_id in frame["sentence_id"].unique():
    stop_counter = 0
    for word in frame[frame["sentence_id"] == sentence_id]["word"]:
        if word == '.':
            stop_counter += 1
    if stop_counter > 1:
        extra_sentences_id.append((sentence_id, stop_counter - 1))
        count_extra_sentences += 1

print(count_extra_sentences)
print(extra_sentences_id)

print(frame[frame["sentence_id"] == 371])

Creating fragments 

In [3]:
corpus = Path('./files/corpuses/eng_crime_and_puhishment.base.zip')
fragments_builder = FragmentsBuilder(corpus, file_name="eng_cip_fragments", localizator=EnglishLocalizator())
fragments_builder.construct_fragments_json()

In [ ]:
with open(Path('./fragments/ru_books_fragments.json')) as json_file:
    json_data = json.load(json_file)

    for fragment in json_data['fragments']:
        print(fragment['text'])

In [3]:
corpus = Path('./files/corpuses/eng_crime_and_puhishment.base.zip')
corpus_reader = CorpusReader(corpus)

frame = corpus_reader.get_frames().skip(2).first()

print(frame["paragraph_id"].values[0])

116225


open-chat-api

In [ ]:
import requests

url = "http://127.0.0.1:5000/v1/chat/completions"

headers = {
    "Content-Type": "application/json"
}

history = []
json_file = open(Path('./fragments/martin_fragments.json'))
json_data = json.load(json_file)
json_file.close()

for fragment in json_data['fragments']:
    print(fragment['text'])
    user_message = fragment['text']
    history.append({"role": "user", "content": user_message})
    data = {
        "model": "openchat_3.5.Q5_K_M.gguf",
        "messages": history
    }

    response = requests.post(url, headers=headers, json=data, verify=False)
    assistant_message = response.json()['choices'][0]['message']['content']
    history.append({"role": "assistant", "content": assistant_message})
    print(assistant_message)
    fragment['retell'] = assistant_message

with open(Path('./fragments/martin_fragments.json'), 'w') as json_file:
    json.dump(json_file)